Importando as bibliotecas

In [2]:
import csv
import fitz
import io
import json
import os
import pymongo
import requests
import wave
from google.cloud import speech
from google.cloud import storage
from google.cloud import vision
from google.oauth2 import service_account
from pydub import AudioSegment

C:\Users\isabella.silveira\AppData\Roaming\Python\Python311\site-packages\pydub\utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)


In [18]:
client_file = r'C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\sa_speech_demo.json'
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)

In [ ]:
def detect_text(image_path):
    # Configurar a variável de ambiente para apontar para suas credenciais de chave de API
    os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r""

    client = vision.ImageAnnotatorClient()
    with io.open(image_path, "rb") as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.text_detection(image=image)
    texts = response.text_annotations
    detected_text = [text.description for text in texts]

    return detected_text

# Caminho para o arquivo de imagem .jpg
image_path = r""

# Chamar a função para detectar texto na imagem
detected_text = detect_text(image_path)

# Imprimir o texto detectado
print("Texto detectado na imagem:")
for text in detected_text:
    print(text)


Texto detectado na imagem:
HEADER DA PÁGINA:
Contém informações que são apresentadas
no topo da página. Geralmente possui dados
da organização e os links mais gerais do site,
como página inicial, mapa do site, etc.
NAV:
É uma área definida para os
principais atalhos da página.
FOOTER DA PÁGINA:
Contém informações que são apresentadas
no rodapé da página. Normalmente tem
dados de contato da organização ou alguma
outra informação relevante.
https://ww
Governo do Brasil
ASSUNTOS
Cidadania
Cultura
PROGRAMA
REDUÇÃO D
Para ministros do (
incentivo irá estimu
Defesa
Economia
Educação
uma nova geração
empreendedores.
Emprego
Esporte
Governo
Ações vão benet
50 mil crianças e
Justiça
Saúde
Segurança
Turismo
Acesso à informação
A intenção é afasta
crime e diminuir a v
e em toda a região
meio de diversas at
HEADER
DA
PÁGINA
:
Contém
informações
que
são
apresentadas
no
topo
da
página
.
Geralmente
possui
dados
da
organização
e
os
links
mais
gerais
do
site
,
como
página
inicial
,
mapa
do
site
,
etc.


In [35]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r''

def upload_to_gcs(bucket_name, source_file_name, destination_blob_name):
    """Uploads a file to the bucket."""
    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(destination_blob_name)
    blob.upload_from_filename(source_file_name)
    print(f"File {source_file_name} uploaded to {destination_blob_name}.")

bucket_name = "isabella_ia_teste"
source_file_name = r""
destination_blob_name = "audio_mono.wav"

upload_to_gcs(bucket_name, source_file_name, destination_blob_name)


File C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\audio_mono.wav uploaded to audio_mono.wav.


In [26]:
client_file = r'C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\sa_speech_demo.json'
credentials = service_account.Credentials.from_service_account_file(client_file)
client = speech.SpeechClient(credentials=credentials)

In [40]:
# Definir a variável de ambiente para o caminho do arquivo de credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = r''

def converter_stereo_para_mono(caminho_audio, caminho_saida):
    audio = AudioSegment.from_wav(caminho_audio)
    audio = audio.set_channels(1)
    audio.export(caminho_saida, format="wav")

def obter_taxa_amostragem_audio(caminho_audio):
    with wave.open(caminho_audio, 'rb') as wf:
        return wf.getframerate()

def fazer_upload_para_gcs(nome_bucket, caminho_arquivo_origem, nome_blob_destino):
    """Faz upload de um arquivo para o bucket."""
    cliente_storage = storage.Client()
    bucket = cliente_storage.bucket(nome_bucket)
    blob = bucket.blob(nome_blob_destino)
    blob.upload_from_filename(caminho_arquivo_origem)
    print(f"Arquivo {caminho_arquivo_origem} enviado para {nome_blob_destino}.")

def transcrever_audio_gcs(uri_gcs, taxa_amostragem_hertz, timeout=600):
    cliente = speech.SpeechClient()
    audio = speech.RecognitionAudio(uri=uri_gcs)
    config = speech.RecognitionConfig(
        encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
        sample_rate_hertz=taxa_amostragem_hertz,
        language_code='pt-BR'  # ou 'en-US' para inglês
    )
    operacao = cliente.long_running_recognize(config=config, audio=audio)
    print('Aguardando a conclusão da operação...')
    resposta = operacao.result(timeout=timeout)
    transcricao = ""
    for resultado in resposta.results:
        transcricao += resultado.alternatives[0].transcript
    return transcricao

nome_bucket = "isabella_ia_teste"
caminho_arquivo_origem = r"C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\Dica-do-professor.wav"
caminho_arquivo_saida = r"C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\audio_mono.wav"
nome_blob_destino = "audio_mono.wav"

# Converter o arquivo de áudio para mono
converter_stereo_para_mono(caminho_arquivo_origem, caminho_arquivo_saida)

# Obter a taxa de amostragem do arquivo de áudio convertido
taxa_amostragem_hertz = obter_taxa_amostragem_audio(caminho_arquivo_saida)

# Fazer o upload do arquivo de áudio para o GCS
fazer_upload_para_gcs(nome_bucket, caminho_arquivo_saida, nome_blob_destino)

# Construir o URI do GCS
uri_gcs = f"gs://{nome_bucket}/{nome_blob_destino}"

# Transcrever o áudio do GCS com um tempo limite maior
transcricao = transcrever_audio_gcs(uri_gcs, taxa_amostragem_hertz, timeout=600)
print(transcricao)


Arquivo C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\audio_mono.wav enviado para audio_mono.wav.
Aguardando a conclusão da operação...
Olá seja bem-vindo a dica do professor da unidade de aprendizagem criação de páginas web com HTML 5 Neste vídeo vamos falar sobre formatação de texto e âncoras em HTML a especificação do HTML5 disponibiliza uma série de elementos para formatar textos e criaram coisas em páginas web para isso você deve usar tags específicas que permitem formatar o celular visando o equilíbrio do código com a semântica do HTML5 dessa dica criamos uma página HTML que apresenta uma relação de músicas de Roberto Carlos nesta página Montamos um minuto que contém links para algumas páginas do Astro objetivo desses links é posicionada a página no início de cada música sem precisar percorrer todo o documento vejamos agora estrutura dess

In [47]:
# Conectar ao MongoDB (certifique-se de ter um servidor MongoDB em execução)
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["mydatabase"]
collection = db["pdf_collection"]

def extract_text_from_pdf(pdf_path):
    # Abrir o documento PDF
    document = fitz.open(pdf_path)

    text = ""
    # Extrair texto de cada página
    for page_num in range(len(document)):
        page = document.load_page(page_num)
        text += page.get_text()

    return text

def index_pdf_text(pdf_path):
    # Extrair texto do PDF
    pdf_text = extract_text_from_pdf(pdf_path)

    # Dividir o texto em palavras-chave (aqui você pode usar técnicas mais avançadas de processamento de linguagem natural)
    keywords = pdf_text.split()

    # Estruturar os dados para o MongoDB
    pdf_data = {
        "pdf_path": pdf_path,
        "keywords": keywords
    }

    # Inserir os dados no MongoDB
    collection.insert_one(pdf_data)
    print(f"Documento PDF indexado: {pdf_path}")

def search_keywords_in_pdf(keyword):
    # Realizar busca no MongoDB
    results = collection.find({"keywords": {"$regex": f".*{keyword}.*", "$options": "i"}})

    # Mostrar os resultados encontrados
    print(f"Resultados para a palavra-chave '{keyword}':")
    for result in results:
        print(result["pdf_path"])

# Exemplo de uso
if __name__ == "__main__":
    # Caminho para o arquivo PDF
    pdf_file_path = r"C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\Capítulo do Livro.pdf"
    # Indexar o texto do PDF no MongoDB
    index_pdf_text(pdf_file_path)

    # Realizar uma busca por palavra-chave no MongoDB
    search_keywords_in_pdf("HTML5")

Documento PDF indexado: C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\Capítulo do Livro.pdf
Resultados para a palavra-chave 'HTML5':
C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\Capítulo do Livro.pdf
C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\Capítulo do Livro.pdf
C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de Trabalho\GenAI\challenge-artificial-intelligence\resources\challenge-artificial-intelligence\resources\Capítulo do Livro.pdf
C:\Users\isabella.silveira\OneDrive - SUPPORTE ARM VENDAS E LOG INTEGRADA LTDA\Área de 

In [91]:
# Texto fornecido
texto = '''Olá seja bem-vindo a dica do professor da unidade de aprendizagem criação de páginas web com HTML 5 Neste vídeo vamos falar sobre formatação de texto e âncoras em HTML a especificação do HTML5 disponibiliza uma série de elementos para formatar textos e criaram coisas em páginas web para isso você deve usar tags específicas que permitem formatar o celular visando o equilíbrio do código com a semântica do HTML5 dessa dica criamos uma página HTML que apresenta uma relação de músicas de Roberto Carlos nesta página Montamos um minuto que contém links para algumas páginas do Astro objetivo desses links é posicionada a página no início de cada música sem precisar percorrer todo o documento vejamos agora estrutura dessa página a instrução doctype HTML e forma navegador que estamos trabalhando com a versão 5 Roberto Carlos é o nosso rei é o título da nossa página ele é exibido na barra de títulos do navegador Carlos é o título principal apresentado na nossa página e se destaque se dá através de H2 nave é uma tag específica do HTML5 tela definir uma área da nossa página onde colocamos os principais links dela sexo definir uma área na nossa página nós podemos ter várias sessões na nossa página Ártico é um alho principal da nossa sessão é nela que vamos colocar as músicas de Roberto Carlos música de Roberto Carlos possui um título esse título é destacado com o que forma uma letra um pouquinho menor que o título principal da nossa página que eu H2 para apresentar as músicas utilizamos a tag pré ela define a formatação tal como está digitada na nossa página ela respeita quebra de linha e linhas em branco ao final de cada música colocamos um atalho para topo esse atalho da direcionar a página para o início mesmo indicador topo no título músicas de Roberto Carlos Esse Ramos o fechamento da tag pré e em seguida iniciamos a próxima música é assim por diante até que todas as músicas esteja página voltando para nossa página podemos ver como ela se comporta clicando no início de uma música percebemos que o navegador posição a Logo no início dela de cada música temos o atalho que volta para o início dela podemos navegar entre as músicas e voltar para o início da página isso com cada uma delas o like de cada música é feito com âncoras ou seja são endereços contatos para dentro da própria página nesse caso como é grande aponta para o início da música Como é grande meu amor por você nesse caso nós temos Esse identificador como é grande com esse título que aponta para o início dessa música assim encerramos a dica do professor que tratam sobre formatação Âncora HTML Espero que você tenha aproveitado essa dica Bons estudos'''

# Nome do arquivo CSV de saída
arquivo1_csv = 'saida.csv'

# Escrever o texto em um arquivo CSV
with open(arquivo1_csv, 'w', newline='', encoding='utf-8') as f:
    escritor_csv = csv.writer(f)
    escritor_csv.writerow([texto])  # Escreve o texto inteiro em uma única célula

print(f'Texto foi salvo em {arquivo1_csv}')


Texto foi salvo em saida.csv


In [92]:

# Estrutura fornecida
dados = {'texto': [
    'HEADER DA PÁGINA:\nContém informações que são apresentadas\nno topo da página. Geralmente possui dados\nda organização e os links mais gerais do site,\ncomo página inicial, mapa do site, etc.\nNAV:\nÉ uma área definida para os\nprincipais atalhos da página.\nFOOTER DA PÁGINA:\nContém informações que são apresentadas\nno rodapé da página. Normalmente tem\ndados de contato da organização ou alguma\noutra informação relevante.\nhttps://ww\nGoverno do Brasil\nASSUNTOS\nCidadania\nCultura\nPROGRAMA\nREDUÇÃO D\nPara ministros do (\nincentivo irá estimu\nDefesa\nEconomia\nEducação\numa nova geração\nempreendedores.\nEmprego\nEsporte\nGoverno\nAções vão benet\n50 mil crianças e\nJustiça\nSaúde\nSegurança\nTurismo\nAcesso à informação\nA intenção é afasta\ncrime e diminuir a v\ne em toda a região\nmeio de diversas at',
    'HEADER', 'DA', 'PÁGINA', ':', 'Contém', 'informações', 'que', 'são', 'apresentadas', 'no', 'topo', 'da', 'página', '.', 'Geralmente', 'possui', 'dados', 'da', 'organização', 'e', 'os', 'links', 'mais', 'gerais', 'do', 'site', ',', 'como', 'página', 'inicial', ',', 'mapa', 'do', 'site', ',', 'etc.', 'NAV', ':', 'É', 'uma', 'área', 'definida', 'para', 'os', 'principais', 'atalhos', 'da', 'página', '.', 'FOOTER', 'DA', 'PÁGINA', ':', 'Contém', 'informações', 'que', 'são', 'apresentadas', 'no', 'rodapé', 'da', 'página', '.', 'Normalmente', 'tem', 'dados', 'de', 'contato', 'da', 'organização', 'ou', 'alguma', 'outra', 'informação', 'relevante', '.', 'https', ':', '//', 'ww', 'Governo', 'do', 'Brasil', 'ASSUNTOS', 'Cidadania', 'Cultura', 'PROGRAMA', 'REDUÇÃO', 'D', 'Para', 'ministros', 'do', '(', 'incentivo', 'irá', 'estimu', 'Defesa', 'Economia', 'Educação', 'uma', 'nova', 'geração', 'empreendedores', '.', 'Emprego', 'Esporte', 'Governo', 'Ações', 'vão', 'benet', '50', 'mil', 'crianças', 'e', 'Justiça', 'Saúde', 'Segurança', 'Turismo', 'Acesso', 'à', 'informação', 'A', 'intenção', 'é', 'afasta', 'crime', 'e', 'diminuir', 'a', 'v', 'e', 'em', 'toda', 'a', 'região', 'meio', 'de', 'diversas', 'at'
]}

# Nome do arquivo CSV de saída
arquivo_csv = 'saida1.csv'

# Escrever os dados no arquivo CSV
with open(arquivo_csv, 'w', newline='', encoding='utf-8') as f:
    escritor_csv = csv.writer(f)
    escritor_csv.writerow(dados['texto'])  # Escreve os dados como uma linha no CSV

print(f'Dados foram salvos em {arquivo_csv}')


Dados foram salvos em saida1.csv


In [167]:

# Texto estruturado
texto = """
DESENVOLVIMENTO 
DE SISTEMAS PHP 
Maurício de Oliveira Saraiva 
Criação de páginas web 
com HTML5
Objetivos de aprendizagem
Ao final deste texto, você deve apresentar os seguintes aprendizados:
 Definir a estrutura de uma página web com HTML5.
 Aplicar a formatação de texto em uma página web com HTML5.
 Desenvolver listas e tabelas em uma página web com HTML5.
Introdução
Sistemas desenvolvidos para a plataforma web, tanto para páginas es-
táticas como dinâmicas, utilizam a linguagem HTML para a exibição 
do conteúdo. Com HTML5, é possível criar páginas web com diversos 
recursos para a apresentação de dados, por meio de novas marcações 
que permitem o uso de semântica e acessibilidade. Isso facilita a pesquisa 
por motores de busca automática, dispositivos próprios para deficientes 
visuais e auditivos, entre outros.
Neste capítulo, você vai estudar a estrutura de páginas web, a for-
matação de texto em documentos hipertexto e apresentação de links, 
listas e tabelas em HTML5.
Estrutura de uma página web com HTML5
HTML, do inglês hyper text markup language, ou linguagem de marcação 
de hipertexto, é uma linguagem utilizada para criar documentos para a web. 
Esses documentos, que também são chamados de páginas, podem conter 
marcações que permitem visualizar e acessar conteúdos web em geral, como 
textos, imagens, áudios e vídeos entre outros (SILVA, 2015).
A versão 5 do HTML, ou simplesmente HTML5, foi publicada em 2014 
pelo World Wide Web Consortium (W3C). W3C é uma comunidade interna-
cional que possui diversos participantes colaborando para o desenvolvimento 
de padrões da web, como as especificações HTML, XML, CSS, etc. (CON-
SÓRCIO..., c2011).
Saiba mais sobre o W3C e as especificações mantidas 
pela comunidade acessando o endereço a seguir 
(CONSÓRCIO…, c2011).
https://goo.gl/VHQ5n
A exibição de uma página HTML pode ser feita por um programa de 
computador chamado navegador web ou simplesmente navegador ou 
browser. 
Existem diversos navegadores disponíveis no mercado, como Chrome, 
Internet Explorer, Firefox, Safari, Opera, Edge, entre outros. No entanto, é 
preciso ter cuidado com as marcações inseridas nas páginas, uma vez que, 
devido às diversas versões existentes dos navegadores, alguns conteúdos 
podem não ser exibidos corretamente, caso o navegador do usuário esteja 
desatualizado.
As marcações de uma página HTML representam mais que a apresentação 
visual do conteúdo. Ao acessar um conteúdo web, você consegue identificar os 
elementos visualmente, pela sua formatação, como títulos em letras maiores, 
dados de cabeçalho e rodapé, idioma, áreas da página e outros.
Contudo, a informação na web não é acessada apenas por pessoas, já que 
o conteúdo pode ser lido por diversos meios, como motores de busca automá-
tica, dispositivos próprios para deficientes visuais e auditivos, equipamentos
portáteis com diferentes tamanhos de tela, etc. (EIS; FERREIRA, 2012).
Com base nisso, no HTML5 foram incluídos alguns elementos que 
permitem a leitura e a interpretação da estrutura de uma página web de 
forma organizada e segura. Essa organização é definida por elementos 
semânticos, que dão significado aos objetos de uma página, conforme 
você verá a seguir.
Criação de páginas web com HTML5
2
Principais elementos da estrutura de uma página HTML5
Como dito anteriormente, uma página HTML é composta por marcações. Essas 
marcações estão inseridas no documento dentro de itens específicos chamados 
de tags, etiquetas ou elementos, que respeitam uma estrutura hierárquica bem 
definida. A Figura 1 apresenta essa estrutura.
Figura 1. Estrutura básica de uma página HTML5.
Doctype
A instrução DOCTYPE, apresentada na linha 1, é um comando especial que 
indica ao navegador a versão do documento HTML. Nas versões anteriores ao 
HTML5, era necessário informar o número da versão, contudo, na versão 5, 
isso não é mais necessário, bastando informar <!DOCTYPE html> para que 
os navegadores a interpretem como uma página HTML5.
Html
A tag <html>, indicada na linha 2, representa a marcação de nível mais 
elevado de uma página HTML. Essa tag envolve outras tags que pertencem 
à estrutura básica de um documento HTML, como <head> e <body>, que 
serão apresentadas na sequência. Essas tags devem ser “fechadas” com suas 
respectivas tags de encerramento: </head>, </body> e </html>, conforme 
ilustrado nas linhas 6, 8 e 9, respectivamente.
Head
O conteúdo pertencente à tag <head>, expresso entre as linhas 3 e 6, deve 
conter informações que são interpretadas pelo navegador, como o título <ti-
tle>, a codificação do idioma <meta> e a inclusão de arquivos CSS <link>.
3
Criação de páginas web com HTML5
Title
A tag <title>, ilustrada na linha 4, serve para indicar o título da página
que será apresentado na barra de título ou na aba documento do navegador. 
Já a tag <meta>, indicada na linha 5, serve para configurar o idioma de
apresentação de caracteres da página. Nesse caso, charset= “utf-8” 
indica que o navegador deve exibir os caracteres do idioma português, como 
cedilha e acentos.
A linguagem HTML não é case sensitive, como a linguagens C e Java que diferen-
ciam letras maiúsculas de minúsculas. Isso significa que não faz diferença escrever 
as tags em maiúsculo ou minúsculo, desde que os comandos estejam corretos. No 
entanto, alguns comandos são escritos de determinada forma por padrão de escrita. 
Além disso, tanto faz identificar as instruções com aspas simples ou duplas, uma vez 
que os navegadores estão preparados para interpretar os dois modos, por exemplo, 
charset= “utf-8” ou charset= ‘utf-8’.
Body
A tag <body> engloba o corpo do documento HTML, ou seja, é dentro dessa 
tag que você deve inserir todo o conteúdo da página que será exibido pelo 
navegador. No HTML5, a tag <body> ganhou seis novas tags, incluídas para 
organizar e permitir pesquisas semânticas mais eficientes por sistemas de busca 
automatizados: <section>, <header>, <aside>, <nav>, <article> e
<footer>, conforme apresentado na Figura 2.
Figura 2. Estrutura da tag <body> no HTML5.
Criação de páginas web com HTML5
4
Section
O elemento <section> define uma seção ou área da página. Esse elemento 
substitui grande parte das tags <div> e <span> que eram usadas de ma-
neira genérica para a mesma finalidade no HTML4 ou anterior. Uma página 
HTML5 pode conter várias seções, que podem representar as mais diversas 
áreas de um site.
Article
O conteúdo principal de uma página HTML5 deve estar localizado dentro 
da tag <article>. Essa tag representa um bloco que contém o conteúdo 
principal do site, como texto, redação e demais informações.
Header e footer
As tags <header> e <footer> representam os cabeçalhos e rodapés, res-
pectivamente. Essas tags podem pertencer a mais de uma área, como seções 
ou a própria página principal. Os cabeçalhos armazenam informações que se 
localizam no topo da página, enquanto os rodapés armazenam informações 
que se localizam no final da página.
Nav
A tag <nav> define uma seção de links de navegação, que pode incluir 
links para outras seções da página ou para outras páginas do site. Essa 
tag facilita a navegação do usuário pelo conteúdo do site.
Aside
A tag <aside> define uma seção lateral de conteúdo, que pode ser rela-
cionada ao conteúdo principal da página, mas não faz parte do conteúdo 
principal em si. Por exemplo, barras laterais, informações relacionadas, 
sugestões, entre outras.
Criação de páginas web com HTML5
5
Figura 3. Tags de formatação de texto em HTML5.
Criação de páginas web com HTML5
6
Textos não recomendados
As tags <font>, <center> e <big> não são mais recomendadas para a for-
matação de textos na versão HTML5. A tag <font> é um exemplo disso, pois 
essa tag é utilizada para alterar a fonte, a cor do texto e o tamanho da fonte 
de um texto, mas a nova versão da linguagem de marcação hipertexto oferece 
vários outros recursos que atendem às mesmas necessidades.
As tags <center> e <big> também não são mais recomendadas, pois não 
têm relação com a semântica do conteúdo da página. No entanto, você pode 
usar a folha de estilos em cascata (CSS) para realizar essas formatações e 
outras formatações de maneira mais eficiente.
Formatação de texto com estilos e 
propriedades CSS
A folha de estilos em cascata (CSS), ou apenas CSS, é uma linguagem de 
marcação hipertextual que define estilos e formatações de uma página web. 
Essa linguagem está separada do conteúdo do documento HTML e, por isso, 
é possível alterar a aparência de uma página web inteira, bastando modificar 
a folha de estilos.
As propriedades CSS que definem a formatação de um texto incluem a 
cor da fonte, o tamanho da fonte, o alinhamento do texto, o estilo da fonte 
(italiano, negrito, etc.), a entrelinha e o espaçamento de letras, entre outros. 
7
Criação de páginas web com HTML5
Todas as propriedades CSS podem ser aplicadas ao texto dentro de uma 
página web, desde que você conheça a sintaxe correta para definir essas pro-
priedades. A Figura 4 apresenta exemplos de formatação de texto com pro-
priedades CSS.
Figura 4. Exemplos de formatação de texto com propriedades CSS.
Desenvolvimento de listas e tabelas 
em uma página web com HTML5
A especificação HTML5 oferece tags para criar listas e tabelas em uma 
página web. As listas podem ser ordenadas ou não ordenadas, enquanto as 
tabelas organizam informações em células e linhas.
Listas em HTML5
As listas são usadas para organizar itens de maneira ordenada ou não 
ordenada. A tag <ul> é usada para criar listas não ordenadas e a tag <ol> é 
usada para criar listas ordenadas. A Figura 5 apresenta essas tags.
Criação de páginas web com HTML5
8
Figura 5. Tags de listas em HTML5.
As listas não ordenadas são úteis quando a ordem dos itens não importa, 
como em listas de tópicos. Já as listas ordenadas são usadas quando a ordem 
dos itens é importante, como em etapas de um processo.
Tabelas em HTML5
As tabelas são usadas para organizar informações em linhas e colunas. A 
tag <table> é usada para criar uma tabela, a tag <tr> é usada para criar uma 
linha e a tag <td> é usada para criar uma célula dentro da linha. A Figura 6 
apresenta essas tags.
9
Criação de páginas web com HTML5
Figura 6. Tags de tabelas em HTML5.
A tag <table> é usada para criar uma tabela, a tag <tr> é usada para criar 
uma linha e a tag <td> é usada para criar uma célula dentro da linha.
Conclusão
Este capítulo apresentou os conceitos básicos sobre a criação de páginas 
web com HTML5. Aprendeu a estrutura de uma página HTML5, a formatação 
de texto e a criação de listas e tabelas. Com esses conhecimentos, você está 
preparado para avançar no desenvolvimento de aplicações web, utilizando 
as novas marcações que permitem a criação de páginas mais acessíveis e 
organizadas.
Atividades propostas
1. Pesquise na internet os navegadores que suportam a versão 5 do HTML.
2. Crie uma página web com HTML5 contendo um título, um parágrafo e uma 
lista de tópicos.
3. Desenvolva uma tabela em HTML5 com, no mínimo, três colunas e três linhas.
Referências
EIS, P. N. C.; FERREIRA, L. P. Desenvolvimento de Aplicações para Web com 
HTML5, CSS3 e JavaScript. 2. ed. São Paulo: Pearson, 2012.
SILVA, M. HTML5 e CSS3: Domine a web do futuro. São Paulo: Novatec, 2015.
"""

# Dividindo o texto em seções principais
sections = texto.split('\n\n')

# Definindo os campos principais
fields = ['title', 'objective', 'introduction', 'structure_html', 'formatting_text', 'lists_tables', 'conclusion', 'references']

# Criando uma lista de dicionários para cada seção
data = []
for i, field in enumerate(fields):
    if i < len(sections):
        data.append({field: sections[i]})

# Escrevendo os dados em um arquivo CSV
csv_file = 'html5_text_sections.csv'
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=fields)
    writer.writeheader()
    writer.writerows(data)

print(f'Arquivo CSV "{csv_file}" criado com sucesso.')


Arquivo CSV "html5_text_sections.csv" criado com sucesso.


In [174]:
class LangChain:
    def __init__(self, conversations):
        self.conversations = conversations

    def generate_response(self, user_input):
        for conversation in self.conversations:
            for pattern in conversation["patterns"]:
                if pattern.lower() in user_input.lower():
                    return conversation["responses"][0]
        return "Desculpe, não tenho uma resposta para essa pergunta."

# Definindo o conjunto de dados com as conversas estruturadas
conversations = [
    {
        "id": "estrutura_html5",
        "patterns": [
            "Quais são os elementos principais de uma página HTML5?",
            "Quais tags compõem a estrutura básica de uma página HTML5?"
        ],
        "responses": [
            "A estrutura básica de uma página HTML5 inclui as seguintes tags principais: <!DOCTYPE html>, <html>, <head>, <title>, <meta>, <link>, <body>, <section>, <header>, <article>, <aside>, <nav>, <footer>."
        ]
    },
    {
        "id": "formatacao_texto_html5",
        "patterns": [
            "Como é feita a formatação de texto em HTML5?",
            "Quais tags são recomendadas para formatação de texto em HTML5?"
        ],
        "responses": [
            "A formatação de texto em HTML5 é realizada utilizando propriedades CSS. As tags não recomendadas para formatação de texto são <font>, <center> e <big>. Em vez disso, deve-se usar CSS para estilizar o texto."
        ]
    },
    {
        "id": "criacao_listas_html5",
        "patterns": [
            "Como criar listas em uma página HTML5?",
            "Quais tags são usadas para criar listas em HTML5?"
        ],
        "responses": [
            "Em HTML5, listas podem ser criadas usando <ul> para listas não ordenadas e <ol> para listas ordenadas. Essas tags permitem organizar itens de forma estruturada."
        ]
    },
    {
        "id": "criacao_tabelas_html5",
        "patterns": [
            "Como criar tabelas em HTML5?",
            "Quais são as tags principais para criar tabelas em HTML5?"
        ],
        "responses": [
            "Tabelas em HTML5 são criadas usando as tags <table> para definir a tabela, <tr> para linhas e <td> para células dentro das linhas. Essas tags permitem organizar informações em formato tabular."
        ]
    },
    {
        "id": "elemento_doctype_html5",
        "patterns": [
            "Para que serve a instrução <!DOCTYPE html> em uma página HTML5?",
            "Qual a função da tag <!DOCTYPE html> em HTML5?"
        ],
        "responses": [
            "A instrução <!DOCTYPE html> informa ao navegador que estamos trabalhando com a versão 5 do HTML."
        ]
    },
    {
        "id": "tag_nav_html5",
        "patterns": [
            "Para que serve a tag <nav> em HTML5?",
            "Como utilizar a tag <nav> em uma página HTML5?"
        ],
        "responses": [
            "A tag <nav> é usada em HTML5 para definir uma área da página onde colocamos os principais links de navegação."
        ]
    },
    {
        "id": "tag_section_html5",
        "patterns": [
            "Qual a função da tag <section> em HTML5?",
            "Como utilizar a tag <section> em uma página HTML5?"
        ],
        "responses": [
            "A tag <section> é usada para definir uma área da página e podemos ter várias seções na mesma página."
        ]
    },
    {
        "id": "tag_article_html5",
        "patterns": [
            "Para que serve a tag <article> em HTML5?",
            "Como utilizar a tag <article> em uma página HTML5?"
        ],
        "responses": [
            "A tag <article> é usada para definir o conteúdo principal de uma seção, como artigos ou postagens de blog."
        ]
    },
    {
        "id": "tag_h2_html5",
        "patterns": [
            "Qual a função da tag <h2> em HTML5?",
            "Como utilizar a tag <h2> em uma página HTML5?"
        ],
        "responses": [
            "A tag <h2> é usada para definir títulos secundários, que são destacados na página com uma letra um pouco menor que a do título principal <h1>."
        ]
    },
    {
        "id": "tag_pre_html5",
        "patterns": [
            "Como usar a tag <pre> em HTML5?",
            "Para que serve a tag <pre> em HTML5?"
        ],
        "responses": [
            "A tag <pre> é usada para definir texto pré-formatado, o que significa que ela preserva espaços em branco, quebras de linha e outros caracteres de formatação como digitados no código fonte."
        ]
    },
    {
        "id": "ancoras_html5",
        "patterns": [
            "Como criar âncoras em HTML5?",
            "Qual a função das âncoras em HTML5?"
        ],
        "responses": [
            "Âncoras são usadas para criar links internos em uma página HTML, permitindo a navegação para diferentes partes da mesma página. Elas são definidas com a tag <a> e o atributo href, usando identificadores de elementos (ID) como destinos."
        ]
    },
    {
        "id": "titulo_pagina_html5",
        "patterns": [
            "Para que serve a tag <title> em HTML5?",
            "Qual a função da tag <title> em uma página HTML5?"
        ],
        "responses": [
            "A tag <title> define o título da página, que é exibido na barra de títulos do navegador."
        ]
    },
    {
        "id": "header_pagina_html5",
        "patterns": [
            "Para que serve a tag <header> em HTML5?",
            "Qual a função do header em uma página HTML5?",
            "O que colocar no <header> de uma página HTML5?",
            "Quais informações devem estar no <header> em HTML5?"
        ],
        "responses": [
            "A tag <header> contém informações que são apresentadas no topo da página. Geralmente possui dados da organização e os links mais gerais do site, como página inicial, mapa do site, etc."
        ]
    },
    {
        "id": "footer_pagina_html5",
        "patterns": [
            "Para que serve a tag <footer> em HTML5?",
            "Qual a função do footer em uma página HTML5?",
            "O que colocar no <footer> de uma página HTML5?",
            "Quais informações devem estar no <footer> em HTML5?"
        ],
        "responses": [
            "A tag <footer> contém informações que são apresentadas no rodapé da página. Normalmente tem dados de contato da organização ou alguma outra informação relevante."
        ]
    }
]

# Inicializando o objeto LangChain com as conversas definidas
langchain = LangChain(conversations)

# Função para interação com o usuário
def responder_pergunta(pergunta):
    resposta = langchain.generate_response(pergunta)
    return resposta

# Exemplo de uso
pergunta_usuario = input("Digite sua pergunta sobre HTML5: ")
resposta = responder_pergunta(pergunta_usuario)
print(resposta)


A tag <footer> contém informações que são apresentadas no rodapé da página. Normalmente tem dados de contato da organização ou alguma outra informação relevante.
